In [6]:
# ------------------------------------------------------------
#  Step 1: Import all required modules and initialize all tools
# ------------------------------------------------------------
from bs4 import BeautifulSoup as soup
from splinter import Browser
import pandas as pd

# initialize splinter
browser = Browser('chrome')
html = browser.html

In [7]:
# ------------------------------------------------------------
#  Step 2: Scrape Nasa Mars News website
# ------------------------------------------------------------
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)



In [12]:
webpage = soup(html, 'html.parser')
webpage.prettify()

'<html xmlns="http://www.w3.org/1999/xhtml">\n <head>\n </head>\n <body>\n  <pre style="word-wrap: break-word; white-space: pre-wrap;"></pre>\n </body>\n</html>'